In [36]:
#AI-TECHGYM-1-7-A-2
#教師なし学習 アソシエーション分析

import pandas as pd
import urllib.request as req

%matplotlib inline

#githubからファイルをDownloadできない場合は以下を実行
#url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
#req.urlretrieve(url, "Online_Retail.xlsx")
#trans = pd.read_excel('Online_Retail.xlsx', sheet_name='Online Retail')
#trans.to_csv("./Online_Retail.csv")

#購買データの読み込み
trans = pd.read_csv('Online_Retail.csv')

#####前処理#####
#キャンセルデータと不明なデータを除くための処理をする

# InoivceNoの先頭1文字をcancel_flgとして追加
trans['cancel_flg'] = trans.InvoiceNo.map(lambda x:str(x)[0])

# cancel_flgでグルーピングして集計
trans.groupby('cancel_flg').size()

#有効なデータに上書きする
trans = trans[(trans.cancel_flg == '5') & (trans.CustomerID.notnull())]
################

#すべてのInvoiceNoをtrans_allとして抽出
trans_ALL = set(trans.InvoiceNo) 
print("すべての購買データ　　　　　 :",end='')
print(len(trans_ALL))

#商品85123Aを購入
trans_X = set(trans[trans['StockCode']=='85123A'].InvoiceNo)
print("商品85123Aを購入　　　　　　 :",end='')
print(len(trans_X))

#商品47566を購入したデータ
trans_Y = set(trans[trans['StockCode']=='47566'].InvoiceNo)
print("商品47566を購入　　　　　　　:",end='')
print(len(trans_Y))

#商品85123Aかつ商品47566を購入したデータをtrans_XYとする
trans_XY = trans_X & trans_Y
print("商品85123Aかつ商品47566を購入:",end='')
print(len(trans_XY))


すべての購買データ　　　　　 :18536
商品85123Aを購入　　　　　　 :1978
商品47566を購入　　　　　　　:1380
商品85123Aかつ商品47566を購入:279


In [19]:
trans.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5


In [35]:
print('すべての購買データ : {}'.format(len(set(trans['InvoiceNo']))))
print('商品 85123A を購入したバスケット数 : {}'.format(len(set(trans[trans['StockCode'] == '85123A']['InvoiceNo']))))
print('商品 47566 を購入したバスケット数 : {}'.format(len(set(trans[trans['StockCode'] == '47566']['InvoiceNo']))))
print('商品 85123A かつ商品 47566 を購入したバスケット数 : {}'.format(len(set(trans[trans['StockCode'] == '85123A']['InvoiceNo']) & set(trans[trans['StockCode'] == '47566']['InvoiceNo']))))

すべての購買データ : 18536
商品 85123A を購入したバスケット数 : 1978
商品 47566 を購入したバスケット数 : 1380
商品 85123A かつ商品 47566 を購入したバスケット数 : 279


In [26]:
type(trans['StockCode'])

pandas.core.series.Series